In [ ]:
#Import packages
#from polygon.rest import RESTClient
import datetime as dt
from dateutil.relativedelta import relativedelta
import pandas as pd
import requests
import numpy as np

#Import API keys from file
polygon_api_key = open("polygon_io_api_key.txt").read()
alphavantage_api_key = open("alphavantage_api_key.txt").read()

#Ticker
ticker = "".upper()

today = dt.datetime.now().date().strftime("%Y-%m-%d")
two_years_ago = dt.datetime.now().date() - dt.timedelta(days=(365*5))

# Calls the API to get historical stock data
stock_url = f'https://api.massive.com/v2/aggs/ticker/{ticker}/range/1/day/{two_years_ago}/{today}?adjusted=true&sort=asc&apiKey={polygon_api_key}'
stock_r = requests.get(stock_url)
stock_data = stock_r.json()

earnings_url = f"https://api.polygon.io/benzinga/v1/earnings?ticker={ticker}&limit=100&sort=date.desc&apiKey={polygon_api_key}&date_status=confirmed"
earnings_r = requests.get(earnings_url)
earnings_data = earnings_r.json()

In [ ]:
##############
# Stock Data #
##############

for i in range(0,len(stock_data['results'])):
    timestamp = (stock_data['results'][i]['t']/1000) + 86400
    stock_data['results'][i]['t'] = dt.datetime.fromtimestamp(timestamp).date().strftime("%Y-%m-%d")

time_series = stock_data['results']
df = pd.DataFrame.from_dict(time_series)
df['t'] = pd.to_datetime(df['t'])

#Earnings
df_len = []
for i in range(len(earnings_data["results"])):
    df_len.append(i)

benzinga_earnings = pd.DataFrame(earnings_data["results"], index=df_len)
df_column_drop = ['actual_eps','estimated_eps','previous_eps','eps_surprise_percent','revenue_surprise_percent','currency','date_status','benzinga_id','importance','company_name','estimated_revenue','previous_revenue','last_updated','eps_surprise','eps_method','actual_revenue','revenue_surprise','revenue_method',]
benzinga_earnings.drop(df_column_drop, axis='columns', inplace=True)

noon = dt.time(12,0,0)
market_periods = []

benzinga_earnings.loc[:, "time"] = pd.to_datetime(benzinga_earnings["time"], format='%H:%M:%S').dt.time

for time in benzinga_earnings["time"]:
    if time >= noon:
        market_periods.append('after-market')
    else:
        market_periods.append('pre-market')

benzinga_earnings.loc[:,'market_period'] = market_periods
benzinga_earnings.drop('time',axis="columns", inplace=True)
benzinga_earnings = benzinga_earnings.head(25)

# Function to adjust the date based on reportTime
def adjust_date(row):
   if row['market_period'] == 'after-market':
       return df[df["t"] > row['date']].min(axis=0)["t"]
   elif row['market_period'] == 'pre-market':
       return df[df["t"] < row['date']].max(axis=0)["t"]
   else:
       return row['date']

benzinga_earnings['date'] = pd.to_datetime(benzinga_earnings['date'])

# Apply the function to create a new column with adjusted dates
# Last earnings is used to determine how the stock moved historically around earnings
benzinga_earnings['adjustedDate'] = benzinga_earnings.apply(adjust_date, axis=1)

#############
# merged_df #
#############

#merged_df joins the first set of earnings data
merged_df = pd.merge(benzinga_earnings, df, left_on='date', right_on="t", how='left')
columns_to_drop = ['t','v','vw']
merged_df.drop(columns=columns_to_drop, inplace=True)
merged_df = merged_df.rename(columns={
   'o': 'report open',
   'h': 'report high',
   'l': 'report low',
   'c': 'report close'
})

###############
# merged_2_df #
###############

#merged_2_df is the final joined dataset that combines the first set of earnings data with the second set of earnings data
merged_2_df = pd.merge(merged_df, df, left_on='adjustedDate', right_on="t", how='left')

merged_2_df.drop(columns=columns_to_drop, inplace=True)

merged_2_df = merged_2_df.rename(columns={
   'o': 'adjusted open',
   'h': 'adjusted high',
   'l': 'adjusted low',
   'c': 'adjusted close'
})

################
# calculations #
################

# used to calculate the difference between the high and low depending is earnings was pre-market or post-market
# and if the stock went up or down

# Initialize a new column with NaN values to calculate the difference between high and low
merged_2_df['diff'] = np.nan

# Iterate over each row and apply the conditions
for index, row in merged_2_df.iterrows():
   #Post-market
   if row['market_period'] == 'after-market':
       if row['report close'] > row['adjusted open']:
           merged_2_df.at[index, 'diff'] = round(row['adjusted low'] - row['report high'],2)
       elif row['report close'] < row['adjusted open']:
           merged_2_df.at[index, 'diff'] = round(row['adjusted high'] - row['report low'],2)
       else:
           merged_2_df.at[index, 'diff'] = np.nan
   #Pre-market
   elif row['market_period'] == 'pre-market':
       if row['adjusted close'] > row['report open']:
           merged_2_df.at[index, 'diff'] = round(row['report low'] - row['adjusted high'],2)
       elif row['adjusted close'] < row['report open']:
           merged_2_df.at[index, 'diff'] = round(row['report high'] - row['adjusted low'],2)
       else:
           merged_2_df.at[index, 'diff'] = np.nan

# Initialize a new column with NaN values for the percentage calculation
merged_2_df['percent_diff'] = np.nan

# Iterate over each row and apply the conditions
for index, row in merged_2_df.iterrows():
   if row['market_period'] == 'after-market':
       if row['report close'] > row['adjusted open']:
           merged_2_df.at[index, 'percent_diff'] = round((row['adjusted low'] / row['report high'])-1,3)
       elif row['report close'] < row['adjusted open']:
           merged_2_df.at[index, 'percent_diff'] = round(row['adjusted high'] / row['report low']-1,3)
       else:
           merged_2_df.at[index, 'percent_diff'] = np.nan
   elif row['market_period'] == 'pre-market':
       if row['adjusted close'] > row['report open']:
           merged_2_df.at[index, 'percent_diff'] = round(row['report low'] / row['adjusted high']-1,3)
       elif row['adjusted close'] < row['report open']:
           merged_2_df.at[index, 'percent_diff'] = round(row['report high'] / row['adjusted low']-1,3)
       else:
           merged_2_df.at[index, 'percent_diff'] = np.nan

merged_2_df.sort_values(by="percent_diff", ascending=False)